In [ ]:
import pandas as pd
import numpy as np
df_raw = pd.read_csv('./data/features.csv')
df_raw.head()

In [ ]:
df = df_raw.copy()
x = df[['returns_1m_30', 'volatility_1m_30', 'mdd_1m_30',
       'skewness_1m_30', 'kurtosis_1m_30', 'returns_1m_180',
       'volatility_1m_180', 'mdd_1m_180', 'skewness_1m_180', 'kurtosis_1m_180',
       'returns_5m_30', 'volatility_5m_30', 'mdd_5m_30', 'skewness_5m_30',
       'kurtosis_5m_30', 'returns_5m_180', 'volatility_5m_180', 'mdd_5m_180',
       'skewness_5m_180', 'kurtosis_5m_180', 'returns_15m_30',
       'volatility_15m_30', 'mdd_15m_30', 'skewness_15m_30', 'kurtosis_15m_30',
       'returns_15m_180', 'volatility_15m_180', 'mdd_15m_180',
       'skewness_15m_180', 'kurtosis_15m_180', 'returns_1h_30',
       'volatility_1h_30', 'mdd_1h_30', 'skewness_1h_30', 'kurtosis_1h_30',
       'returns_1h_180', 'volatility_1h_180', 'mdd_1h_180', 'skewness_1h_180',
       'kurtosis_1h_180']]
y = df['stopping_returns_1m_60']
x_columns = x.columns

# Convert the timestamp from milliseconds to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Sort by timestamp to ensure correct time-based splitting
df = df.sort_values(by='timestamp')

# Define the number of weeks for training and testing
train_weeks = 8
test_weeks = 1

# Calculate the start and end dates
latest_date = df['timestamp'].max()
splits = []


# Generate 8 splits for cross-validation
for i in range(8):
    test_end_date = latest_date - pd.Timedelta(weeks=7 - i)
    test_start_date = test_end_date - pd.Timedelta(weeks=test_weeks)
    train_end_date = test_start_date
    train_start_date = df['timestamp'].min()

    train_set = df[(df['timestamp'] >= train_start_date) & (df['timestamp'] < train_end_date)]
    test_set = df[(df['timestamp'] >= test_start_date) & (df['timestamp'] < test_end_date)]

    splits.append((train_set, test_set))

# Display summary of the splits
split_summary = []
for i, (train, test) in enumerate(splits):
    print({
        'Split': i+1,
        'Train Start': train['timestamp'].min(),
        'Train End': train['timestamp'].max(),
        'Test Start': test['timestamp'].min(),
        'Test End': test['timestamp'].max(),
        'Train Size': len(train),
        'Test Size': len(test),
    })

In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram of y
plt.figure(figsize=(10, 6))
plt.hist(y, bins=30, color='blue', alpha=0.7, edgecolor='black')
plt.title('Histogram of Stopping Returns (y)', fontsize=16)
plt.xlabel('Stopping Returns', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import QuantileRegressor
from ngboost import NGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_pinball_loss
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
import statsmodels.api as sm
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions, keeping 95% variance


imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
pca = PCA(n_components=0.95)

q = 0.95

def quantile_loss(preds, dtrain): # xgboost custom objective function
    labels = dtrain.get_label()
    errors = labels - preds
    grad = np.where(errors > 0, -q, 1 - q)
    hess = np.ones_like(grad)
    return grad, hess

models = {
    'QuantileRegressor_alpha_0': QuantileRegressor(quantile=q, alpha=0.0, fit_intercept=True),
    'QuantileRegressor_alpha_0_05': QuantileRegressor(quantile=q, alpha=0.05, fit_intercept=True),
    'QuantileRegressor_alpha_0_50': QuantileRegressor(quantile=q, alpha=0.50, fit_intercept=True),
    'NGBRegressor': NGBRegressor(n_estimators=100, verbose=False),

    'GrandientBoostingRegressor': GradientBoostingRegressor(loss='quantile', alpha=q, n_estimators=100)
    
}

cross_validation_results = {
    model_name: [] for model_name in models.keys()
}

for i, (train, test) in enumerate(splits):
    print(f'Split {i + 1}, Train Size: {len(train)}, Test Size: {len(test)}')
    imputer = SimpleImputer(strategy='mean')
    x_train_imputed = imputer.fit_transform(train[x_columns])
    x_test_imputed = imputer.transform(test[x_columns])

    x_train_scaled = scaler.fit_transform(x_train_imputed)
    x_test_scaled = scaler.transform(x_test_imputed)

    # Generate interaction terms using PolynomialFeatures (degree=2 for pairwise interactions)
    x_train_interaction = poly.fit_transform(x_train_scaled)
    x_test_interaction = poly.transform(x_test_scaled)

    # add constant for quantile regression
    x_train_interaction_df = pd.DataFrame(x_train_interaction, columns=poly.get_feature_names_out(input_features=x_columns))
    x_test_interaction_df = pd.DataFrame(x_test_interaction, columns=poly.get_feature_names_out(input_features=x_columns))
    x_train_interaction_df = sm.add_constant(x_train_interaction_df)
    x_test_interaction_df = sm.add_constant(x_test_interaction_df)

    x_train_pca = pca.fit_transform(x_train_interaction_df)
    x_test_pca = pca.transform(x_test_interaction_df)

    # Convert to DataFrame
    x_train_pca_df = pd.DataFrame(x_train_pca, columns=[f'PC{i+1}' for i in range(x_train_pca.shape[1])])
    x_train_pca_df = sm.add_constant(x_train_pca_df)

    x_test_pca_df = pd.DataFrame(x_test_pca, columns=[f'PC{i+1}' for i in range(x_test_pca.shape[1])])
    x_test_pca_df = sm.add_constant(x_test_pca_df)


    y_train = train['stopping_returns_1m_60']
    y_test = test['stopping_returns_1m_60']

    for model_name, model in models.items():
        model.fit(x_train_pca_df, y_train)
        if model_name == 'NGBRegressor':
            y_pred_dist = model.pred_dist(x_test_pca_df)
            y_pred = y_pred_dist.ppf(q)
        else:
            y_pred = model.predict(x_test_pca_df)

        # Calculate the mean absolute error
        loss_test = mean_pinball_loss(y_test, y_pred, alpha=0.0)
        cross_validation_results[model_name].append(loss_test)
        print(f'Model: {model_name}, Loss: {loss_test:.4f}')
# Display the cross-validation results
cross_validation_results_df = pd.DataFrame(cross_validation_results)
cross_validation_results_df.describe()





In [ ]:
from sklearn.linear_model import QuantileRegressor


# Fit Quantile Regression for the 5th and 95th quantiles
quantiles = [5, 95]
quantile_models = {}

for q in quantiles:
    model = QuantileRegressor(quantile=q / 100.0, alpha=0, solver='highs')
    model.fit(X_train, y[train_index])
    quantile_models[q] = model

# Extract and display the summary of the models
summary_5 = quantile_models[5].summary()
summary_95 = quantile_models[95].summary()

summary_5, summary_95

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract coefficients and p-values for both quantiles
coef_5 = quantile_models[5].params[1:]  # Exclude intercept
pvals_5 = quantile_models[5].pvalues[1:]

coef_95 = quantile_models[95].params[1:]
pvals_95 = quantile_models[95].pvalues[1:]

# Filter significant coefficients (p < 0.05)
significant_5 = coef_5[pvals_5 < 0.05]
significant_95 = coef_95[pvals_95 < 0.05]

# Plot for 5th quantile
plt.figure(figsize=(12, 6))
plt.barh(significant_5.index, significant_5.values, color='blue', alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Significant Features in 5th Quantile Regression (Extreme Negative Returns)')
plt.grid(axis='x')
plt.gca().invert_yaxis()
plt.show()

# Plot for 95th quantile
plt.figure(figsize=(12, 6))
plt.barh(significant_95.index, significant_95.values, color='green', alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Significant Features in 95th Quantile Regression (Extreme Positive Returns)')
plt.grid(axis='x')
plt.gca().invert_yaxis()
plt.show()

# Calculate model efficiency using Pseudo R-squared
pseudo_r2_5 = quantile_models[5].prsquared
pseudo_r2_95 = quantile_models[95].prsquared

pseudo_r2_5, pseudo_r2_95

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Generate interaction terms using PolynomialFeatures (degree=2 for pairwise interactions)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interaction = poly.fit_transform(X_scaled)

# Convert to DataFrame for easier analysis
interaction_feature_names = poly.get_feature_names_out(x_columns)
X_interaction_df = pd.DataFrame(X_interaction, columns=interaction_feature_names)

# Add constant for quantile regression
X_interaction_df = sm.add_constant(X_interaction_df)

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions, keeping 95% variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_interaction)

# Convert to DataFrame
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

# Add constant for quantile regression
X_pca_df = sm.add_constant(X_pca_df)

# Align with y_train
X_pca_train = X_pca_df.iloc[y_train.index]

# Fit Quantile Regression with PCA components
pca_models = {}
for q in quantiles:
    model = sm.QuantReg(y_train, X_pca_train).fit(q=q)
    pca_models[q] = model

# Extract and display the summary of the models
pca_summary_5 = pca_models[0.05].summary()
pca_summary_95 = pca_models[0.95].summary()

pca_summary_5, pca_summary_95

In [ ]:
# Extract significant components for visualization
pvals_pca_5 = pca_models[0.05].pvalues[1:]
pvals_pca_95 = pca_models[0.95].pvalues[1:]

coef_pca_5 = pca_models[0.05].params[1:]
coef_pca_95 = pca_models[0.95].params[1:]

# Filter significant coefficients (p < 0.05)
significant_pca_5 = coef_pca_5[pvals_pca_5 < 0.05]
significant_pca_95 = coef_pca_95[pvals_pca_95 < 0.05]

# Plot for 5th quantile
plt.figure(figsize=(12, 6))
plt.barh(significant_pca_5.index, significant_pca_5.values, color='blue', alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Significant PCA Components in 5th Quantile Regression (Extreme Negative Returns)')
plt.grid(axis='x')
plt.gca().invert_yaxis()
plt.show()

# Plot for 95th quantile
plt.figure(figsize=(12, 6))
plt.barh(significant_pca_95.index, significant_pca_95.values, color='green', alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Significant PCA Components in 95th Quantile Regression (Extreme Positive Returns)')
plt.grid(axis='x')
plt.gca().invert_yaxis()
plt.show()